<a href="https://colab.research.google.com/github/dla9944/God_damn_deeplearning/blob/master/NLP_test_Shakespear_RNN_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 셰익스피어 데이터 다운로드

In [1]:
import string
all_chars = string.printable
vocab_size = len(all_chars)
vocab_dict = dict((c,i) for (i,c) in enumerate(all_chars))

# Reference
* https://github.com/karpathy/char-rnn/tree/master/data/tinyshakespeare
* 원데이터 작성자는 TESLA, OpenAi Director 출신 사이언티스트로 현재는 유튜브에서도 연구를 만나볼수 있습니다.(https://www.youtube.com/@AndrejKarpathy)

In [2]:
def str2ints(s, vocab_dict):
    return [vocab_dict[c] for c in s]

In [3]:
def ints2str(x, vocab_array):
    return "".join([vocab_array[i] for i in x])

In [4]:
from google.colab import files
uploaded = files.upload()

Saving input.txt to input.txt


In [5]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import tqdm

In [26]:
class ShakespeareDataset(Dataset):
   def __init__(self, path, chunk_size = 200):
       data = str2ints(open(path).read().strip(), vocab_dict)

       data = torch.tensor(data, dtype = torch.int64).split(chunk_size)

       if len(data[-1]) < chunk_size:
          data = data[:-1]
       self.data = data
       self.n_chunks = len(self.data)

   def __len__(self):
       return self.n_chunks
   def __getitem__(self, idx):
       return self.data[idx]

In [27]:
ds = ShakespeareDataset('/content/input.txt', chunk_size = 200)

loader = DataLoader(ds, batch_size = 32, shuffle = True, num_workers = 4)

In [28]:
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from torch.optim.adam import Adam

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(f'running in {device}')

running in cuda


In [29]:
# model 신경망
class SequenceGenerationNet(nn.Module):

   def __init__(self, num_embeddings, embedding_dim = 50, hidden_size = 50, num_layers = 1, dropout = 0.2):
       super().__init__()
       self.emb = nn.Embedding(num_embeddings, embedding_dim)
       self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first = True,
                           dropout=dropout)
       self.linear = nn.Linear(hidden_size, num_embeddings)
   def forward(self, x, h0 = None):
       x = self.emb(x)
       x, h = self.lstm(x, h0)
       x = self.linear(x)
       return x, h

In [30]:
# 문장 생성하는 함수

def generate_seq(net, start_phrase = 'The King said ', length = 200, temperature = 0.8, device = device):
    # 평가모드
    net.eval()
    result = []

    start_tensor = torch.tensor(str2ints(start_phrase, vocab_dict),
                                dtype = torch.int64).to(device)
    # 선두에 batch 차원
    x0 = start_tensor.unsqueeze(0)

    # RNN을 통해서 출력과 새로운 내부 상태 추출
    o, h = net(x0)

    # 출력을 확률로 변환 (exp() =지수함수)
    out_dist = o[:, -1].view(-1).exp()

    # 확률로부터 실제 문자의 인덱스 샘플링
    top_i = torch.multinomial(out_dist, 1)[0]

    # save
    result.append(top_i)

    # 결과는 RNN에 투입
    for i in range(length):
        inp = torch.tensor([[top_i]], dtype = torch.int64)
        inp = inp.to(device)
        o, h = net(inp, h)
        out_dist = o.view(-1).exp()
        top_i = torch.multinomial(out_dist,1)[0]
        result.append(top_i)

    # 시작 문자열과 생성된 문자열을 모아서 반환
    return start_phrase + ints2str(result, all_chars)

In [31]:
from statistics import mean

In [ ]:
# 모델 훈련용 함수

net = SequenceGenerationNet(vocab_size, 20, 50, num_layers = 2, dropout = 0.1)
net.to('cuda:0')

# optimizer function = 아담
optim = optim.Adam(net.parameters())

# loss function = 크로스 엔트로피
loss_function = nn.CrossEntropyLoss()

In [32]:
# 훈련 시작

for epoch in range(50):
    net.train()
    losses = []
    for data in tqdm(loader):
        x = data[:,:-1]
        # y는 두번째부터 마지막 문자까지
        y = data[:, 1:]
        x = x.to('cuda:0')
        y = y.to('cuda:0')
        y_pred, _ = net(x)

        # batch와 step축을 통합해서 crossentropyloss에 전달
        loss = loss_function(y_pred.view(-1, vocab_size), y.view(-1))
        net.zero_grad()
        loss.backward()
        optim.step()
        losses.append(loss.item())
    print(f'에포크 : [{epoch}]. loss 평균 : [{mean(losses)}]')
    with torch.no_grad():
         print(generate_seq(net, device = 'cuda:0'))

  0%|          | 0/175 [00:00<?, ?it/s]

에포크 : [0]. loss 평균 : [4.622120701926095]
The King said 2|RCZDqTVHP(u94DU?%kpRu/zZ7Epx8MAFIJ&+^J?ct&N,bj?*Oh?=RK*np,%Jz4{ %G;!kOxdGV9v}
xe6K/Ia7w&LwB9O5,S<PF3RG5WAOB'cO5Fp'[^TPS^1pT)0X?M;CzEA8nU1P5l+ZbHm4PozcBQ<4d@>XNIL~s,17g_%dak9k4\w[yg31be&LIZ$4|uq00


  0%|          | 0/175 [00:00<?, ?it/s]

에포크 : [1]. loss 평균 : [4.6221133259364535]
The King said gw]00{U+	$VPj[5-FdUEYK 6JWva!uKlF)|Cffc/@#Ml:Mr!ev[-F_m	}[9&PnF#Z{W,j?5k4Z&~(5k%lwD=\|"A?Q0_/k]%d1jcTuD7Igo|;tb-*?UM=cE+diR>eD_0vRoR2(7d]/=/f6Y	b=d,G1NgT"2E5|\| ubHj!,k|$1kp4IZy_it@R;Pd]Ty97,'\4E


  0%|          | 0/175 [00:00<?, ?it/s]

...

에포크 : [47]. loss 평균 : [4.622139538356236]
S=0L%F1Ur^M}<K'+X1dp]e$5OF5Oj}Hx0zOJz`zy31U-J`\-ZKJaL__.NN3ipVr,@C?6iqU?aqC #k[G]jr?)%qpV~.L
YD,\ljBAS%
EkS9[&K


  0%|          | 0/175 [00:00<?, ?it/s]

에포크 : [48]. loss 평균 : [4.622133494785854]
The King said tU>3G}q_b {J)`X3DT2:f-9C&xkTP<.AJxBC{)h	9)*2eIAq#,2ApbxGucH--U~5&sd}DB|JbeMv#Bi|)
sSWOzN\	JZoI5#$
DZB9K]CIRXe0(;xyZTaTG3_/Mj{0^Bf,l/ mq<L]>hl*jzjq	Q
[:8~%,s9*:\;oPgU)#x	q;C:0+)MTt:c}yB6gF{wsK(?	,"OR\>


  0%|          | 0/175 [00:00<?, ?it/s]

에포크 : [49]. loss 평균 : [4.622109421321324]
<6sWB+\s#*q9lFKiU4'Nyqw527];]b_h4W\Z,[Rg\V^}.%cd.sI(.D460X)m7*B#vk<Zom75x
nOi'%h`wU?`,S=zysS]wDY4	!4&O	4rC2\\Uw[aZA\^yml&w


In [16]:
# 훈련 시작
# epoch 100회
for epoch in range(100):
    net.train()
    losses = []
    for data in tqdm(loader):
        x = data[:,:-1]
        # y는 두번째부터 마지막 문자까지
        y = data[:, 1:]
        x = x.to('cuda:0')
        y = y.to('cuda:0')
        y_pred, _ = net(x)

        # batch와 step축을 통합해서 crossentropyloss에 전달
        loss = loss_function(y_pred.view(-1, vocab_size), y.view(-1))
        net.zero_grad()
        loss.backward()
        optim.step()
        losses.append(loss.item())
    print(f'에포크 : [{epoch}]. loss 평균 : [{mean(losses)}]')
    with torch.no_grad():
         print(generate_seq(net, device = 'cuda:0'))

  0%|          | 0/175 [00:00<?, ?it/s]

에포크 : [0]. loss 평균 : [1.6736537953785489]
The King said Done?
You staining nistey not but sis:
'he dom hour if the lingly love my muton.

Dil:
For the huscle such, let it authing and wistraice to in princes
Though youl mite of nisoly Clovess hild witle her 


  0%|          | 0/175 [00:00<?, ?it/s]

에포크 : [1]. loss 평균 : [1.6697360399791172]
The King said not to all strep upon they huch arceed:
Who thin thy resques chane marrmos. I forful!
'Noure, gons argerere viliat rest to you, ine,
Revizer unnock me way! I prince,
Enet of rowing onourseous fromo is 


  0%|          | 0/175 [00:00<?, ?it/s]

...

에포크 : [97]. loss 평균 : [1.5650410693032402]
The King said Henry, preaydether will abis.
Spoke a have my falsuxe you with him,
Spine storb, peace: till not her so mave's execuch
could yew Richange, I come of this faol's-begs.
Let yass additon not, by may Bunge


  0%|          | 0/175 [00:00<?, ?it/s]

에포크 : [98]. loss 평균 : [1.5647880547387258]
The King said coudss you't and confesses.

GLOUCESTER:
Yey come againtess Glouce chastine bail.

ROMEO:
Gou's air.

ESCLARINA:
No, Signance '
How? and a deeds the must as desommonss.

WOTFill:
Ay, that I not live, t


  0%|          | 0/175 [00:00<?, ?it/s]

에포크 : [99]. loss 평균 : [1.563388684817723]
The King said He Lablows, my him.

PETRUCHIO:
Gorlians, come not paigh made you
Age shall his eins tweef his god, lord,
If this abclied to-do break sweet tell God.

PAULINA:
The penife, in mine all!

RICHARD:
I woul
